<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import numpy as np
from pymystem3 import Mystem
m = Mystem()
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import f1_score
import nltk
#from nltk.corpus import stopwords as nltk_stopwords
import spacy
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

## Подготовка

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head

<bound method NDFrame.head of         Unnamed: 0                                               text  toxic
0                0  Explanation\nWhy the edits made under my usern...      0
1                1  D'aww! He matches this background colour I'm s...      0
2                2  Hey man, I'm really not trying to edit war. It...      0
3                3  "\nMore\nI can't make any real suggestions on ...      0
4                4  You, sir, are my hero. Any chance you remember...      0
...            ...                                                ...    ...
159287      159446  ":::::And for the second time of asking, when ...      0
159288      159447  You should be ashamed of yourself \n\nThat is ...      0
159289      159448  Spitzer \n\nUmm, theres no actual article for ...      0
159290      159449  And it looks like it was actually you who put ...      0
159291      159450  "\nAnd ... I really don't think you understand...      0

[159292 rows x 3 columns]>

In [4]:
df.text

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159287    ":::::And for the second time of asking, when ...
159288    You should be ashamed of yourself \n\nThat is ...
159289    Spitzer \n\nUmm, theres no actual article for ...
159290    And it looks like it was actually you who put ...
159291    "\nAnd ... I really don't think you understand...
Name: text, Length: 159292, dtype: object

In [5]:
#удаляю столбец Unnamed:
df = df.drop(['Unnamed: 0'], axis=1)

In [6]:
#ввожу функцию очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [7]:
%%time
#очищаю тексты постов:
df['text'] = df['text'].apply(clear_text) 

CPU times: user 4.55 s, sys: 43.7 ms, total: 4.59 s
Wall time: 4.59 s


In [8]:
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


Произведем очистку данных от разделителей и заглавных символов

In [9]:
#def cleaning(text):
  #  text = re.sub(r"(?:\n|\r)", " ", text)
   # text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    #text = text.lower()
    #return text

# df['text'] = df['text'].apply(cleaning)

In [10]:
print(df.text[:3])

0    explanation why the edits made under my userna...
1    d aww he matches this background colour i m se...
2    hey man i m really not trying to edit war it s...
Name: text, dtype: object


Проведем лемматизацию

In [11]:
#corpus = df['text'].values.astype('U')
#def lemmatize(text):
 #   lemm = m.lemmatize(text)
  #  return "".join(lemm)

#corpus[0] = lemmatize(corpus[0])
#corpus

In [12]:
#ввожу функцию РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#ввожу функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [13]:
%%time
#леммализирую тексты постов:
df['text'] = df['text'].apply(lemm_text) 

CPU times: user 17min 16s, sys: 1min 24s, total: 18min 40s
Wall time: 18min 41s


In [14]:
df.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [15]:
#разделяю выборки в соотношении 80/10/10:
features = df.drop(['toxic'], axis=1) 
target = df.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=12345)

In [16]:
#смотрю размеры выборок:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

(127433, 1)
(127433,)
(15929, 1)
(15929,)
(15930, 1)
(15930,)


In [17]:
#смотрю соотношение 1/0 в выборках на примере target_train:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений 1 в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

Доля значений 1 в тренировочной выборке: 0.10166911239631807


In [18]:
#уменьшаю кол-во 0 в выборках train:

df_train = df.iloc[target_train.index]
target_train_0 = df_train[df_train['toxic'] == 0]['toxic']
target_train_1 = df_train[df_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = df.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

print('Соотношение 1/0 в тренировочной выборке:')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)

Соотношение 1/0 в тренировочной выборке:
0    0.5
1    0.5
Name: toxic, dtype: float64

(25912,)
(25912,)


Выводы: При подготовке данных:

 * Удален ненужный столбец
 * Проведена очиста текстов комментариев от ненужных знаков, леммализировал, убрал стоп-слова
 * Сбалансированы данные в целевом признаке

## Обучение

Список стоп слов

In [19]:
stopwords = set(nltk_stopwords.words('english'))

Разделим выборки

In [20]:
target=df['toxic'].values
features = df['text']
f_other, f_test, t_other, t_test = train_test_split(features, target, test_size = .1, random_state = 42)
f_train, f_valid, t_train, t_valid = train_test_split(f_other, t_other, shuffle=False, test_size=0.25, random_state = 42)

f_train.shape[0], f_valid.shape[0], f_test.shape[0]

(107521, 35841, 15930)

Выполним TFIDF векторизацию

In [21]:

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tfidf_train = count_tf_idf.fit_transform(f_train)
tfidf_valid = count_tf_idf.transform(f_valid)
tfidf_test = count_tf_idf.transform(f_test)

Произведем подбор параметров

In [22]:
%%time
pipe = Pipeline([
    (
    ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
    )
    ])


param_grid = [
        {

            'model': [LogisticRegression(random_state=42, solver='liblinear')],
            'model__penalty' : ['l1', 'l2'],
            'model__C': list(range(1,15,3))
        }
]
grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1', cv=3, verbose=True, n_jobs=-1)
best_grid = grid.fit(tfidf_train, t_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters is: {'model': LogisticRegression(C=4, penalty='l1', random_state=42, solver='liblinear'), 'model__C': 4, 'model__penalty': 'l1'}
Best score is: 0.7642355461017781
CPU times: user 1min 34s, sys: 1min 6s, total: 2min 40s
Wall time: 2min 40s


In [23]:
%%time
params_forest = {
    'n_estimators': list(range(40,150,40)),
    'max_depth':[5,15],
    'max_features' : list(range(1,20, 2))
}


model_forest = RandomForestClassifier(random_state=12345)
                                 
grid = GridSearchCV(model_forest, param_grid=params_forest, scoring='f1', cv=3, verbose=True, n_jobs=-1)
best_grid = grid.fit(tfidf_train, t_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Best parameters is: {'max_depth': 5, 'max_features': 1, 'n_estimators': 40}
Best score is: 0.0
CPU times: user 7min 22s, sys: 4.91 s, total: 7min 26s
Wall time: 7min 27s


Произведем обучение модели регрессии

In [24]:
model = LogisticRegression(random_state=42, C = 4, penalty = 'l1', solver='liblinear', max_iter=200)
model.fit(tfidf_train, t_train)
valid_pred = model.predict(tfidf_valid)
f1_score(t_valid, valid_pred)

0.7832395617411905

In [25]:
test_pred = model.predict(tfidf_test)
print(metrics.classification_report(t_test, test_pred), metrics.f1_score(t_test, test_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     14314
           1       0.87      0.71      0.78      1616

    accuracy                           0.96     15930
   macro avg       0.92      0.85      0.88     15930
weighted avg       0.96      0.96      0.96     15930
 0.7794417971409122


In [26]:
#model_forest = RandomForestClassifier(max_depth=5, max_features=1, n_estimators = 50)
#model_forest.fit(tfidf_train, t_train)
#valid_pred_f = model_forest.predict(tfidf_valid)
#f1_score(t_valid, valid_pred_f)

## Выводы

Необходимая модель найдена, результат полученный на тестовой выборке удовлетворяет условию, в данных не найдены пропуски и дубликаты. Лучшая модель основанна на алгоритме LogisticRegression

F1=0.7832 - модель обучилась с качеством не менее 0.75

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны